### Este notebook tem como função criar uma pasta chamada "Kaggle Faces Dataset" onde os rostos recortados dos vídeos serão guardados

Inicialmente, carregamos as dependências necessárias

In [561]:
import cv2

import pandas as pd

from facenet_pytorch import MTCNN

from PIL import Image
import glob, os

import torch
import torchvision
import numpy as np

import random

import time

import pathlib
from pathlib import Path

import numpy as np

from tqdm.notebook import tqdm

Define-se o device onde será rodada a detecção de rostos.

In [2]:
# Definimos um device onde os tensores estarão sendo processados
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}, {}'.format(device, torch.cuda.get_device_name()))

Running on device: cuda:0, GeForce RTX 2070


In [805]:
a = pd.read_json(folder/"metadata.json").transpose()
a

,label,split,original
hsypgwsufp.mp4,FAKE,train,nbnipejygk.mp4
ntzgbkzofo.mp4,FAKE,train,cqlarprtdy.mp4
ataulynpgd.mp4,FAKE,test,uzrkbzwdvi.mp4
idzntwkkjy.mp4,FAKE,train,lvnjzrvzwy.mp4
rdqokuannd.mp4,FAKE,train,mujubwlspn.mp4
...,...,...,...
fseamiushb.mp4,REAL,train,None
kghcxtmytq.mp4,FAKE,train,cnrqyitpqs.mp4
mzecordeuu.mp4,FAKE,train,dzochfswby.mp4
bkwicglevl.mp4,REAL,train,None


# 1° passo: Criar uma classe que cuida de todas as leituras dos vídeos

Cria-se uma classe que cuida da leitura de todos os vídeos do dataset.

In [824]:
# Cuida de lidar com o acesso aos vídeos e devolver os paths / labels corretamente
class Videos():
    def __init__(self, root="./Kaggle Dataset"):
        # Guarda o folder_path
        self.root = Path(root)
        
        folders = list(self.root.glob("*"))
        order = lambda x: int(x.stem.split("_")[-1])
        self.folders = sorted(folders, key=order)
        
    def getRandomVideo(self):
        # Lê o arquivo JSON que contém as informações dos deepfakes naquela pasta
        
        folder_path = random.choice(self.folders)
        
        metadata = pd.read_json(folder_path/'metadata.json').transpose() # Troca colunas e linhas para ficarEM no formato correto
        
        video_path = random.choice(list(folder_path)) # Devolve um WindowsPath
        
        video_name = video_path.name # videoaleatorio.mp4
        
        label = metadata.loc[video_name].label
        
        return video_path, video_name, label
    
    def getRandomLabeledVideo(self, label):
        if label not in ["FAKE", "REAL"]:
            print("Label deve ser FAKE ou REAL")
            return
        
        folder_path = random.choice(self.folders) # "./Kaggle Dataset/some_folder"
        
        metadata = pd.read_json(folder_path/'metadata.json').transpose() # Troca colunas e linhas para ficarEM no formato correto
        
        fake_videos = metadata[metadata.label == label].index # Filtra os videos que são FAKE

        video_path = folder_path/random.choice(fake_videos)
        
        video_name = video_path.name # videoaleatorio.mp4
        
        label = label
        
        return video_path, video_name, label
        
    def getEquivalentRealVideo(self, video_path):
        
        if not isinstance(video_path, pathlib.WindowsPath):
            video_path = Path(video_path)
            if not video_path.exists():
                print("Video path não existe.")
                return
            
        folder_path = video_path.parent
        
        metadata = pd.read_json(folder_path/'metadata.json').transpose() # Troca colunas e linhas para ficarEM no formato correto
        
        try: 
            video = metadata.loc[video_path.name]
        except:
            print("O vídeo indicado não existe no metadata.")
            return
        
        if video.label == "REAL":
            print("O vídeo em questão já é um vídeo real.")
            return
        
        real_video_name = video.original
        real_video_path = folder_path.joinpath(real_video_name)
        label = "REAL"
        
        return real_video_path, real_video_name, label
    
    def getAllVideosPath(self):
        for video_name, columns in self.metadata.iterrows():
            yield self.folder_path + '/' + video_name, video_name, columns[0] # Label
            
    def generateMetadataTestSplit(self, proportion=0.2): # 20% para o set de treinamento
        print("Ajuste de metadata.json em andamento...")
        np.random.seed(42)
        for folder in self.folders:
            metadata_path = folder/"metadata.json"
            metadata = pd.read_json(metadata_path).transpose()
            if "test" not in metadata.values:
                counts = metadata.label.value_counts()

                reals_quantity_to_test = int(proportion*counts["REAL"])
                fakes_quantity_to_test = int(proportion*counts["FAKE"])

                real_videos = metadata[metadata.label == 'REAL'].index
                fake_videos = metadata[metadata.label == 'FAKE'].index

                reals_to_test = np.random.choice(real_videos, size=reals_quantity_to_test)
                fakes_to_test = np.random.choice(fake_videos, size=fakes_quantity_to_test)

                metadata.loc[reals_to_test, 'split'] = "test"
                metadata.loc[fakes_to_test, 'split'] = "test"
            else:
                print("Pass...")
            #metadata.transpose().to_json(metadata_path)
        print("Finalizado com sucesso!")

Função **showVideo()**: Mostra o vídeo para checagem.

In [801]:
def showVideo(video_path, resize_factor=0.6):
    
    # Captura o vídeo no path
    cap = cv2.VideoCapture(str(video_path))

    while(cap.isOpened()):
        ret, frame = cap.read() # Lê o próximo frame
        if ret: # Sucesso na leitura
            frame = cv2.resize(frame, (int(frame.shape[1]*resize_factor), int(frame.shape[0]*resize_factor)))
            cv2.imshow('frame', frame)
            
            # Apertar a tecla 'q' para sair do vídeo.
            key = cv2.waitKey(25)
            if key == 113:
                break
                
        else:
            break

    cv2.destroyAllWindows()
    cap.release()

In [802]:
def showVideoWithDetection(video_path, label=None, padding=0, size=-1, separate_face_box=False, resize_factor=0.6):
    
    # Captura o vídeo no path
    cap = cv2.VideoCapture(str(video_path))
    
    # Configura a cor a ser colocada na LABEL
    if label == 'REAL':
        color = (0, 255, 0) # Verde
    elif label == "FAKE":
        color = (0, 0, 255) # Vermelho  
    else:
        color = (0, 255, 255)
    
    face = None
    
    while(cap.isOpened()):
        ret, frame = cap.read() # Lê o próximo frame
        if ret: # Sucesso na leitura
            boxes, _ = mtcnn.detect(Image.fromarray(frame)) # Detecta as imagens. O método detect só aceita numpy arrays
            if boxes is not None: # Só entra se rostos forem detectados
                for box in boxes: # Para cada uma das bouding boxes encontradas em um único frame (a princípio só deve ter uma)
                    box = [int(b) for b in box]
                    if separate_face_box:
                        face = frame[int(box[1] - padding):int(box[3] + padding), int(box[0] - padding):int(box[2] + padding)].copy()
                        if face is not None:
                            if size > 0:
                                face = cv2.resize(face, (size, size))
                            cv2.imshow('face', face)
                    cv2.putText(img=frame, text=label, org=(box[0], box[1]), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=color, thickness=2)
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), color=[0, 255, 0], thickness=5)

            frame = cv2.resize(frame, (int(frame.shape[1]*resize_factor), int(frame.shape[0]*resize_factor)))
            cv2.imshow('frame', frame)
            
            # Apertar a tecla 'q' para sair do vídeo.
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        else:
            break
    
    cv2.destroyAllWindows()
    cap.release()

# 2° passo: Observar alguns vídeos e ajustar o threshold do MTCNN para a detecção de rostos

**MTCNN** - *Multi-task Cascaded Convolutional Network*

Ajustamos os thresholds e utilizamos a função de mostrar os vídeos, verificando visualmente se ele se comporta bem na maioria dos casos.

In [803]:
# Margin não faz diferença se o método .detect() for utilizado
IMAGE_SIZE = 224
MARGIN = 0
MIN_FACE_SIZE = 90
THRESHOLDS = [0.68, 0.75, 0.80]
POST_PROCESS = False
SELECT_LARGEST = True
KEEP_ALL = False
DEVICE = device

# ----------------------------------

mtcnn = MTCNN(image_size=IMAGE_SIZE,
              margin=MARGIN, 
              min_face_size=MIN_FACE_SIZE, 
              thresholds=THRESHOLDS,
              post_process=POST_PROCESS,
              select_largest=SELECT_LARGEST, 
              keep_all=KEEP_ALL, 
              device=device)

### Objeto Videos

Definimos nosso objeto vídeos com o diretório onde contém as pastas do conjunto de dados.

In [825]:
# Instaciamos uma objeto da classe Videos
videos = Videos(root="./Kaggle Dataset")

#### Vídeo Real Aleatório

Para cancelar a visualização do vídeo em tempo real, apertar a tecla `q` do teclado.Para cancelar a visualização do vídeo em tempo real, apertar a tecla `q` do teclado.

In [818]:
video_path, video_name, label = videos.getRandomLabeledVideo(label="REAL")

# Podemos passar um padding para verificar a quantidade desejada de recorte ao redo do rosto detectado
showVideo(video_path)
showVideoWithDetection(video_path, label=label)

#### Vídeo Falso Aleatório

Para cancelar a visualização do vídeo em tempo real, apertar a tecla `q` do teclado.

In [828]:
video_path, video_name, label = videos.getRandomLabeledVideo(label="FAKE") # Recupera o caminho de um vídeo aleatório na pasta que esteja com a label 'FAKE'

showVideo(video_path)
showVideoWithDetection(video_path, label=label)

#### Vídeo Falso Aleatório e sua versão REAL

Para cancelar a visualização do vídeo em tempo real, apertar a tecla `q` do teclado.

In [820]:
video_path, video_name, label = videos.getRandomLabeledVideo(label="FAKE") # Recupera o caminho de um vídeo aleatório na pasta que esteja com a label 'FAKE'
video_path_real, video_name_real, label_real = videos.getEquivalentRealVideo(video_path)

showVideo(video_path)
showVideo(video_path_real)

Tudo estando ok, agora podemos separar todos os vídeos em um conjunto de treinamento para realizar a validação cruzada (80%) e um conjunto de vídeo finais para teste (20%).

In [583]:
videos.generateMetadataTestSplit(proportion=0.2)

Ajuste de metadata.json em andamento...
Finalizado com sucesso!


# 3° Passo: Criar uma classe que recebe um vídeo, detecta todos os rostos, rescorta eles e os salva no diretório em questão

Classe **CropSaver()**: Recorta todos os rostos de um vídeo dada uma taxa de checagem por frame.

In [901]:
class CropSaver(Videos):
    
    def __init__(self, root="./Kaggle Dataset"):
        super().__init__(root)
        # Instaciamos a MTCNN de forma interna
        IMAGE_SIZE = 224
        MARGIN = 0
        MIN_FACE_SIZE = 90
        THRESHOLDS = [0.68, 0.75, 0.80]
        POST_PROCESS = False
        SELECT_LARGEST = True
        KEEP_ALL = False
        DEVICE = device

        # ----------------------------------

        self.mtcnn = MTCNN(image_size=IMAGE_SIZE,
                      margin=MARGIN, 
                      min_face_size=MIN_FACE_SIZE, 
                      thresholds=THRESHOLDS,
                      post_process=POST_PROCESS,
                      select_largest=SELECT_LARGEST, 
                      keep_all=KEEP_ALL, 
                      device=device).eval()

    def callCropFaces(self, path_to_save="./Kaggle Faces Dataset", call="test-video"):
        path_to_save = Path(path_to_save)
        if call == "test-video":
            video_path = list(self.folders[0].glob("*.mp4"))[0]
            metadata = pd.read_json(self.folders[0]/"metadata.json").transpose()
            video_name = video_path.name
            label = metadata.loc[video_name].label
            split = metadata.loc[video_name].split
            print("-"*20 + "Beggining..." + "-"*20 )
            begin = time.time()
            self.saveCropFaces(video_path, path_to_save, split, label)
            end = time.time()
            elapsed = end - begin
            print(f"Iteration Complete | Time elapsed: {elapsed // 60}m{elapsed%60}s")
        elif call == "test-folder":
            videos_path = list(self.folders[0].glob("*.mp4"))
            metadata = pd.read_json(self.folders[0]/"metadata.json").transpose()
            print("-"*20 + "Beggining..." + "-"*20 )
            begin = time.time()
            for video_path in videos_path:
                video_name = video_path.name
                label = metadata.loc[video_name].label
                split = metadata.loc[video_name].split
                self.saveCropFaces(video_path, path_to_save, split, label)
            end = time.time()
            elapsed = end - begin
            print(f"Iteration Complete | Time elapsed: {elapsed // 60}m{elapsed%60}s")

    def saveCropFaces(self, video_path, path_to_save, split, label, batch_size=20, padding=0, size=-1, check_every_frame=30):

        # Instancia um VideoCapture do arquivo presente em video_path (no caso, o vídeo)
        cap = cv2.VideoCapture(str(video_path))
        
        # Pega, em inteiros, a quantidade de frames do vídeo
        v_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Cria um path para salvar os rostos recortados do vídeo
        class_path = path_to_save.joinpath(split, label) # Path(./Kaggle Faces Dataset/train/FAKE ou REAL)
        
        if not class_path.exists():
            print(f"The folder {class_path} não é alcançável.")
            return
        
        # Inicializa frames como uma lista vazia
        frames = []
        face_count = 0
        
        # Entra num loop que percorre o vídeo até ele acabar
        for _ in range(1, v_len + 1):
            # Realiza um grab() no próximo frame, mas não o decodifica. Isso ajuda a agilizar o processo se não for necessário
            # recuperar todos os frames a todo o momento.
            success = cap.grab()
            # Só recorta o rosto se o frame atual for mod check_every_frame, ou seja, ele só decodifica de check_every_frame em check_every_frame frames.
            if not success:
                continue
            if _ == 1 or _ % check_every_frame == 0:
                success, frame = cap.retrieve()
            else:
                continue
            if not success:
                continue
            # Realiza um append do frame atual na lista frames (ele é capturado no formato BGR porém a MTCNN espera no formato RGB)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = Image.fromarray(frame)
            frames.append(frame)
            
            # Se o tamanho dos frames alcançou o batch_size OU o meu iterador de frames chegou no máixmo juntamento com meu tamanho de frames dentro da lista sendo maior que zero, continua pra capturar o rosto
            if len(frames) >= batch_size or (_ == v_len and len(frames) > 0):
                # Utiliza o MTCNN para detectar todas as bounding boxes de todos os rostos
                boxes, probs = self.mtcnn.detect(frames)
                # Verifica se não foi obtida nenhuma bounding box em todo o batch
                if not all(x is None for x in boxes):
                    # Acessa cada um dos frames no batch
                    for i, boxes_f in enumerate(boxes):
                        # Verifica houve None para o frame atual
                        if boxes_f is not None:
                            # Acessa cada uma das bounding boxes dentro de um único frame (pode haver vários rostos)
                            for bbox in boxes_f:
                                # Obtém o rosto
                                face = frames[i].crop(box=(bbox[0]-padding, 
                                                           bbox[1]-padding, 
                                                           bbox[2]+padding, 
                                                           bbox[3]+padding))

                                # Se desejado, aplica um resize
                                if size > 0:
                                    face = face.resize((size, size))
                                    
                                # face_count serve para não ocorrer sobrescrição de mais de um rosto por frame
                                face_count += 1
                                
                                # Cria o path para o rosto atual
                                
                                path = class_path.joinpath(f"{video_path.parts[-2]} {video_path.parts[-1]} {face_count} .jpg")
                                # Salva o rosto na pasta correta.
                                face.save(path)

                # Após o batch ser aplicado, resetamos a lista
                frames = []

        # Solta o objeto do VideoCapture
        cap.release()

Testamos agora para apenas 1 folder.

In [902]:
cropsaver = CropSaver()

In [903]:
cropsaver.callCropFaces(call="test-video")

--------------------Beggining...--------------------
Kaggle Faces Dataset\test\FAKE\dfdc_train_part_0 aaqaifqrwn.mp4 1 .jpg
Kaggle Faces Dataset\test\FAKE\dfdc_train_part_0 aaqaifqrwn.mp4 2 .jpg
Kaggle Faces Dataset\test\FAKE\dfdc_train_part_0 aaqaifqrwn.mp4 3 .jpg
Kaggle Faces Dataset\test\FAKE\dfdc_train_part_0 aaqaifqrwn.mp4 4 .jpg
Kaggle Faces Dataset\test\FAKE\dfdc_train_part_0 aaqaifqrwn.mp4 5 .jpg
Kaggle Faces Dataset\test\FAKE\dfdc_train_part_0 aaqaifqrwn.mp4 6 .jpg
Kaggle Faces Dataset\test\FAKE\dfdc_train_part_0 aaqaifqrwn.mp4 7 .jpg
Kaggle Faces Dataset\test\FAKE\dfdc_train_part_0 aaqaifqrwn.mp4 8 .jpg
Kaggle Faces Dataset\test\FAKE\dfdc_train_part_0 aaqaifqrwn.mp4 9 .jpg
Kaggle Faces Dataset\test\FAKE\dfdc_train_part_0 aaqaifqrwn.mp4 10 .jpg
Kaggle Faces Dataset\test\FAKE\dfdc_train_part_0 aaqaifqrwn.mp4 11 .jpg
Iteration Complete | Time elapsed: 0.0m0.7239961624145508s


Realizamos um loop utilizando a função saveCropFaces, que será chamada uma vez por vídeo. Com isso, esperamos e conferimos a pasta para ver se os arquivos estão corretamente lá.

In [397]:
BATCH_SIZE = 20
PADDING = 10
SIZE = -1
CHECK_EVERY_FRAME = 15
CHANNEL = None
FOLDER = 'Dataset provisório'
SIZE_FOLDER = 'no-resize-color'

# -------------------------------------------------------------------------
init = time.time()
print("-------------- Início do folder {} --------------".format(folder))
videos_quantity = len(videos.video_files)
percentage = 5
print_every = int(videos_quantity / (100/percentage))
    
for n_video, VIDEO_DATA in enumerate(videos_generator):
    
    saveCropFaces(*VIDEO_DATA, 
                  batch_size=BATCH_SIZE, 
                  padding=PADDING, 
                  size=SIZE, 
                  check_every_frame=CHECK_EVERY_FRAME, 
                  channel=CHANNEL, 
                  folder=FOLDER,
                  size_folder=SIZE_FOLDER)
    
    if n_video % print_every == 0:
        print("{}: {:.2f}%...".format(folder, round(n_video / videos_quantity * 100)))
        
end = time.time()
total = end - init
print("Tempo que levou para a conclusão de {} vídeos: {:.0f}:{:.0f} min".format(len(os.listdir(folder_path)), int(total/60), total % 60))

-------------- Início do folder dfdc_train_part_0 --------------
dfdc_train_part_0: 0.00%...
dfdc_train_part_0: 5.00%...
dfdc_train_part_0: 10.00%...
dfdc_train_part_0: 15.00%...
dfdc_train_part_0: 20.00%...
dfdc_train_part_0: 25.00%...
dfdc_train_part_0: 30.00%...
dfdc_train_part_0: 35.00%...
dfdc_train_part_0: 40.00%...
dfdc_train_part_0: 45.00%...
dfdc_train_part_0: 49.00%...
dfdc_train_part_0: 54.00%...
dfdc_train_part_0: 59.00%...
dfdc_train_part_0: 64.00%...
dfdc_train_part_0: 69.00%...
dfdc_train_part_0: 74.00%...
dfdc_train_part_0: 79.00%...
dfdc_train_part_0: 84.00%...
dfdc_train_part_0: 89.00%...
dfdc_train_part_0: 94.00%...
Tempo que levou para a conclusão de 1335 vídeos: 21:5 min


Observamos que demora em torno de 22 minutos para terminar uma pasta. Depende bastante dq uantidade de vídeos na pasta. Podemos aumentar o batch_size para 30 para acelerar o processo.

# 4° Passo: Agora, podemos finalmente exportar todo o dataset de vídeos em imagens de rostos. (Pode levar algumas horas para concluir).

Definimos nossa MTCNN final

In [7]:
# Margin não faz diferença se o método .detect() for utilizado
IMAGE_SIZE = 224
MARGIN = 0
MIN_FACE_SIZE = 100
THRESHOLDS = [0.78, 0.78, 0.78]
POST_PROCESS = False
SELECT_LARGEST = False
KEEP_ALL = True
DEVICE = device

# ----------------------------------

mtcnn = MTCNN(image_size=IMAGE_SIZE,
              margin=MARGIN, 
              min_face_size=MIN_FACE_SIZE, 
              thresholds=THRESHOLDS,
              post_process=POST_PROCESS,
              select_largest=SELECT_LARGEST, 
              keep_all=KEEP_ALL, 
              device=device)

Aqui nós removemos os folder que já foram avaliados em momentos anteriores quando rodou-se o script do código. Para novos foldes será necessário escrever manualmente o nome na lista.

In [8]:
done_folders = ['dfdc_train_part_31', 
                'dfdc_train_part_10', 
                'dfdc_train_part_40', 
                'dfdc_train_part_22', 
                'dfdc_train_part_41', 
                'dfdc_train_part_21', 
                'dfdc_train_part_38', 
                'dfdc_train_part_36', 
                'dfdc_train_part_43']

recent_folders = [folder for folder in folders if folder not in done_folders]

In [9]:
pick_order = np.random.RandomState(seed=42).permutation(len(recent_folders))
pick_order

array([19, 16, 15, 26,  4, 12, 37, 27, 39,  6, 25,  9, 13, 31, 34,  8, 17,
       24,  0, 33,  5, 11,  1, 29, 21,  2, 30, 36,  3, 35, 23, 32, 10, 22,
       18, 20,  7, 14, 28, 38])

E iniciamos o loop que pode levar de algumas horas até alguns dias para terminar, dependendo do hardware. Valores importante que devem ser setados nessa etapa por fim:
- `CHECK_EVERY_FRAME`: Define de quantos em quantos frames será realizada a inferência da rede para obter o rosto. Caso o valor seja 1 a rede realizará a inferência de 1 em 1 frame, ou seja, tentará encontrar e recortar os rostos de todos os frames de todos os vídeos.
- `PADDING`: Controla a margem de recorte das imagens. Quanto maior este número, maior será a margem em volta do rosto recortado.
- `SIZE`: Controla as dimensões de saída do rosto recortado. Caso seja `-1`, o rosto será salvo nas dimensões originais obtidas. Como muitos frameworks de Deep Learning contém transformações do tipo **resize** extremamente otimizados, preferi manter o tamanho originalmente obtido permitindo maior flexibilidade com quem venha a implementar a leitura dessas imagens futuramente.
- `BATCH_SIZE`: Controla o tamanho do batch de frames que será processado de uma única vez pela MTCNN. Um maior número significa mais paralelado que significa mais rápido, porém pode ser que a memória dos hardwares não suporte valores muito elevados. Exemplo: Minha RTX 2070 com 8GB de memória suporta um máximo batch_size em torno de 30.
- `CHANNEL`: Controla o espaço de cores do rosto recortado. Caso seja `None`, será salva a imagem recortada do rosto com os 3 canais originais RGB. A única outra implementação disponível aqui é `'luma'`, que transforma a imagem para o espaço YCrCb e salva somente o canal Y (luma).
- `FOLDER`: Nome da pasta onde estará a subpasta que contenha as pastas "FAKE" e "REAL".
- `SIZE_FOLDER`: Nome da subpasta onde estarão as pastas "FAKE" e "REAL", que a função utilizará para distribuir as imagens corretamente.

In [22]:
PATH = './Kaggle Dataset/'
CHECK_EVERY_FRAME = 15
PADDING = 10
SIZE = -1
BATCH_SIZE = 20
CHANNEL = None
FOLDER = 'Faces Dataset'
SIZE_FOLDER = 'no-resize-color'

# -------------------------------------------------------------------------

for pick in pick_order:
    folder_path = PATH + recent_folders[pick]
    # Instaciamos uma objeto da classe Videos aplicado ao path obtido
    videos = Videos(folder_path)
    videos_generator = videos.getAllVideosPath()    
    
    print("-------------- Início do folder {} --------------".format(folders[pick]))
    videos_quantity = len(videos.video_files)
    percentage = 25
    print_every = int(videos_quantity / (100/percentage))
    
    init = time.time()
    for n_video, VIDEO_DATA in enumerate(videos_generator):
        
        saveCropFaces(*VIDEO_DATA, 
                      batch_size=BATCH_SIZE, 
                      padding=PADDING, 
                      size=SIZE, 
                      check_every_frame=CHECK_EVERY_FRAME, 
                      channel=CHANNEL, 
                      folder=FOLDER,
                      size_folder=SIZE_FOLDER)
        
        if n_video % print_every == 0:
            print("{}: {:.2f}%...".format(recent_folders[pick], round(n_video / videos_quantity * 100)))
        
        
    end = time.time()
    total = end - init
    print("Tempo para a conclusão do diretório: {:.0f}:{:.0f} min".format(int(total/60), total % 60))

-------------- Início do folder dfdc_train_part_26 --------------
dfdc_train_part_29: 0.00%...
dfdc_train_part_29: 25.00%...
dfdc_train_part_29: 50.00%...
dfdc_train_part_29: 75.00%...
dfdc_train_part_29: 100.00%...
Tempo para a conclusão do diretório: 40:25 min
-------------- Início do folder dfdc_train_part_23 --------------
dfdc_train_part_26: 0.00%...
dfdc_train_part_26: 25.00%...
dfdc_train_part_26: 50.00%...
dfdc_train_part_26: 75.00%...
dfdc_train_part_26: 100.00%...
Tempo para a conclusão do diretório: 35:15 min
-------------- Início do folder dfdc_train_part_22 --------------
dfdc_train_part_25: 0.00%...
dfdc_train_part_25: 25.00%...
dfdc_train_part_25: 50.00%...
dfdc_train_part_25: 75.00%...
dfdc_train_part_25: 100.00%...
Tempo para a conclusão do diretório: 36:0 min
-------------- Início do folder dfdc_train_part_32 --------------
dfdc_train_part_37: 0.00%...
dfdc_train_part_37: 25.00%...
dfdc_train_part_37: 50.00%...
dfdc_train_part_37: 75.00%...
dfdc_train_part_37: 100.00%

Aqui o processo foi interrompido pois a quantidade total de imagens de rostos recortadas já superava o meio milhão, suficiente para rodar algumas redes.